In [3]:
import os
import numpy as np
from datetime import datetime as dt
import datetime
from datetime import timedelta
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sms
import csv
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn import decomposition
from sklearn import manifold
from functools import reduce
#from sklearn.impute import KNNImputer
import secrets
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
sns.set()

In [ ]:
#@title Import Time Series Feature Extraction Library
import warnings
warnings.filterwarnings('ignore')
!pip install https://github.com/fraunhoferportugal/tsfel/archive/v0.1.2.tar.gz 2>&1
!pip install --upgrade -q gspread >/dev/null 2>&1
!pip install gspread oauth2client >/dev/null 2>&1
!pip install pandas >/dev/null 2>&1
!pip install scipy >/dev/null 2>&1
!pip install novainstrumentation >/dev/null 2>&1
!pip install pandas_profiling >/dev/null 2>&1

In [3]:
!pip install https://github.com/fraunhoferportugal/tsfel/archive/v0.1.2.tar.gz 2>&1

In [24]:
feature = "Durchfluss"
dates = ['16.02','18.02','26.04','28.04']
#dates = ['28.04']
list_dfs_MD = []
list_dfs_data = []
for i in range(len(dates)):
    date = dates[i]  
    orig_folder = 'C:/Users/Anni/CS/_sig_crun/ReLkat/data/data_csv/'+date+"/"
    df_MD = pd.read_csv(os.path.join(orig_folder,"MD_"+feature+"_"+date+".csv"),index_col = 0)
    data_df = pd.read_csv(os.path.join(orig_folder,date+"_total.csv"),index_col = 0)
    list_dfs_MD.append(df_MD)
    list_dfs_data.append(data_df)
    print(date," loading MD for: ",feature," from ", date, "with number of datapoints: ",df_MD.shape[0])
df_MD_total = pd.concat(list_dfs_MD)
df_MD_total.reset_index(drop=True,inplace=True)
print("Total number of datapoints: ",df_MD_total.shape[0])
df_cycle_times = pd.concat(list_dfs_data)
df_cycle_times = df_cycle_times[["start_time","end_time","cycle_time"]]
df_cycle_times['cycle_time'] = (df_cycle_times['cycle_time']).apply(lambda x: str(x)[10:22])
df_cycle_times['cycle_time'] =  (df_cycle_times['cycle_time']).apply(lambda x: datetime.datetime.strptime(x,'%M:%S.%f'))
df_cycle_times['cycle_time'] = (df_cycle_times['cycle_time']).apply(lambda x:datetime.timedelta(minutes=x.minute,seconds=x.second,microseconds=x.microsecond).total_seconds())
mms = MinMaxScaler()
df_cycle_times[['cycle_time']] = mms.fit_transform(df_cycle_times[['cycle_time']])
MD = df_MD_total
MD.sort_values(by=['sys_time'],inplace=True)

16.02  loading MD for:  Durchfluss  from  16.02 with number of datapoints:  12695
18.02  loading MD for:  Durchfluss  from  18.02 with number of datapoints:  46298
26.04  loading MD for:  Durchfluss  from  26.04 with number of datapoints:  68940
28.04  loading MD for:  Durchfluss  from  28.04 with number of datapoints:  133651
Total number of datapoints:  261584


In [25]:
pd.set_option('display.max_rows', 700)
start_time = '2021-02-16 11:55:05.876769333'
end_time = '2021-04-28 18:01:48.559682659'
MD.drop(['uuid','plc_time','sequence_number','value_string','value_bytes','identifier'],axis=1,inplace=True)
MD['start_cycle'] = np.datetime64('NaT')
MD['end_cycle'] = np.datetime64('NaT')
MD['end_cycle'] = np.where((MD['label'].str.contains('Taktzeit 1 Start '))&(MD['value_bool']==False),MD['sys_time'],np.datetime64('NaT'))
MD.loc[(MD['value_bool']==True),"start_cycle"] = MD['sys_time']
MD = MD[(MD['sys_time']>= start_time)&(MD['sys_time']<= end_time)]
MD.sort_values(by=['sys_time'],inplace=True)
MD.reset_index(drop=True,inplace=True)
print(MD.shape)

# MD['sys_time'] = MD['sys_time'].astype('datetime64[s]')

(261583, 7)


In [26]:
MD['cycle'] = MD['value_bool'][~MD['start_cycle'].isnull()]*np.array([i for i in range(MD['start_cycle'].nunique())])
MD['cycle'].fillna(method='ffill',inplace=True)

MD = MD[~((MD['cycle']==3)|(MD['cycle']==4))]
MD.reset_index(drop=True,inplace=True)
MD['cycle'] = MD['value_bool'][~MD['start_cycle'].isnull()]*np.array([i for i in range(MD['start_cycle'].nunique())])
MD['cycle'].fillna(method='ffill',inplace=True)
MD.sort_values(by=['sys_time'],inplace=True)
#grouped = MD.groupby('cycle').size()    #agg('count')    #filter(lambda x: x['Count'].min()>8 )
#grouped

In [27]:
# check nr. of cycles and their length
print(MD['cycle'].nunique())
MD['cycle'].value_counts()

8468


12      2156
8420    1732
29      1727
8432    1673
39      1671
        ... 
4679       1
7720       1
709        1
2532       1
3112       1
Name: cycle, Length: 8468, dtype: int64

In [28]:
# check start index of cycles
MD['value_bool'][~MD['start_cycle'].isnull()]*np.array([i for i in range(MD['start_cycle'].nunique())])

0            0
16           1
32           2
1358         3
2702         4
          ... 
251293    8463
252795    8464
254375    8465
255789    8466
257192    8467
Name: value_bool, Length: 8468, dtype: object

In [29]:
features = list(MD['label'].unique())[1:]
features

['Kühlschmierstoff: Durchfluss ND Vorlauf  in l/min (DB_PROZESSWERTE.iw_450_93 )',
 'Kühlschmierstoff: Durchfluss HD Vorlauf in l/min (DB_PROZESSWERTE.iw_450_92 )']

In [30]:
df_features = []
for feature in features:
    df_feature = MD[MD['label']==feature][['sys_time','value_integer','value_double']]
    df_feature[feature] = np.where(df_feature['value_integer'].isnull(),df_feature['value_double'],df_feature['value_integer'])
    df_feature.drop(['value_integer','value_double'],axis=1,inplace=True)
    df_features.append(df_feature)

In [31]:
df_MD = reduce(lambda left,right: pd.merge(left,right,on=['sys_time'],how='outer'), df_features)
df_MD.sort_values(by=['sys_time'],inplace=True)
df_MD.reset_index(drop=True,inplace=True)

In [32]:
# Load times of start and end for all the cycles from existing df
dates = ['16.02','18.02','26.04','28.04']
list_dfs = []
max_length = 0
for i in range(len(dates)):
    date = dates[i]
    orig_folder = 'C:/Users/Anni/CS/_sig_crun/Relkat/data/data_csv/'+date+"/"
    data_df = pd.read_csv(os.path.join(orig_folder,date+"_total.csv"),index_col = 0)
    data_df = data_df[["start_time","end_time"]]
    list_dfs.append(data_df)
    print(date," loading data from ", date, "with number of datapoints: ",data_df.shape)    
df_total = pd.concat(list_dfs)
df_total.reset_index(drop=True,inplace=True)
print(df_total.shape)
start_cycle = np.array(df_total['start_time'])
end_cycle = np.array(df_total['end_time'])

16.02  loading data from  16.02 with number of datapoints:  (6, 2)
18.02  loading data from  18.02 with number of datapoints:  (27, 2)
26.04  loading data from  26.04 with number of datapoints:  (27, 2)
28.04  loading data from  28.04 with number of datapoints:  (84, 2)
(144, 2)


In [45]:
print(df_MD.shape)
df_MD.loc[0,'Kühlschmierstoff: Durchfluss HD Vorlauf in l/min (DB_PROZESSWERTE.iw_450_92 )']=0
df_MD.fillna(method='ffill',inplace=True)
MD.sort_values(by=['sys_time'],inplace=True)
#df_MD.head(100)

(236563, 3)


In [46]:
df_MD

,sys_time,Kühlschmierstoff: Durchfluss ND Vorlauf in l/min (DB_PROZESSWERTE.iw_450_93 ),Kühlschmierstoff: Durchfluss HD Vorlauf in l/min (DB_PROZESSWERTE.iw_450_92 )
0,2021-02-16 11:55:06.416670285,1.0,0.0
1,2021-02-16 11:55:06.431654429,2.0,0.0
2,2021-02-16 11:55:06.461913834,3.0,0.0
3,2021-02-16 11:55:06.482242402,4.0,0.0
4,2021-02-16 11:55:06.511932771,5.0,0.0
...,...,...,...
236558,2021-04-28 18:01:31.870234970,6.0,0.0
236559,2021-04-28 18:01:31.934427505,5.0,0.0
236560,2021-04-28 18:01:31.965102276,3.0,0.0
236561,2021-04-28 18:01:31.979857696,1.0,0.0


In [34]:
df_ND = df_MD.drop(['Kühlschmierstoff: Durchfluss HD Vorlauf in l/min (DB_PROZESSWERTE.iw_450_92 )'],axis=1)
df_HD = df_MD.drop(['Kühlschmierstoff: Durchfluss ND Vorlauf  in l/min (DB_PROZESSWERTE.iw_450_93 )'], axis=1)

In [14]:
# define number of datapoints that are gonna be used - starting from the beginning of the time series
df_HD_cycles = []
min_len = 1300

for i in range(len(start_cycle)):
    HD_cycle = df_HD[(df_HD['sys_time']>=start_cycle[i])&(df_HD['sys_time']<=end_cycle[i])].drop('sys_time',axis=1).values
    
    #print(len(HD_cycle))
   
    if(len(HD_cycle) > min_len):
        HD_cycle = HD_cycle[:min_len]
        #print(HD_cycle.shape)
    else:
        pad_len = min_len - len(HD_cycle)
        HD_cycle = np.pad(HD_cycle.reshape(len(HD_cycle),), (0,pad_len), 'constant', constant_values=(0))
        #print(HD_cycle.shape)
    df_HD_cycles.append(HD_cycle.reshape(len(HD_cycle),1))
   #print('_______________')   

In [15]:
# Make sure array has time series of equal length for every cycle
ar_DF_KSS_HD = np.vstack(df_HD_cycles)
ar_DF_KSS_HD.shape[0]/1300

144.0

In [16]:
# define number of datapoints that are used - starting from the end of the time series
df_ND_cycles = []
min_len = 800

for i in range(len(start_cycle)):
    ND_cycle = df_ND[(df_ND['sys_time']>=start_cycle[i])&(df_ND['sys_time']<=end_cycle[i])].drop('sys_time',axis=1).values
    
    #print(len(ND_cycle))
    
    if(len(ND_cycle) > min_len):
        ND_cycle = ND_cycle[-min_len:]
        #print(ND_cycle.shape)
    else:
        pad_len = min_len - len(ND_cycle)
        ND_cycle = np.pad(ND_cycle.reshape(len(ND_cycle),), (pad_len,0), 'constant', constant_values=(0))
        #print(ND_cycle.shape)
    df_ND_cycles.append(ND_cycle.reshape(len(ND_cycle),1))
    #print('_______________')      

In [17]:
# Make sure array has time series of equal length for every cycle
ar_DF_KSS_ND = np.vstack(df_ND_cycles)
ar_DF_KSS_ND.shape[0]/800

144.0

In [ ]:
feature = 'DF_KSS_HD'
ar = ar_DF_KSS_HD

dest_folder = "C:/Users/.../MD_feature_arrays/"
filename = "ar_"+feature
np.save(os.path.join(dest_folder,filename)+".npy", ar)
print(os.path.join(dest_folder,filename)+".npy")

In [18]:
def create_feat_array(feat_name,ar,window_size):
    cfg_file_stat = tsfel.get_features_by_domain('statistical')                                         
    ar_feat_stat = tsfel.time_series_features_extractor(cfg_file_stat, ar, window_size=window_size)
    
    # Remove highly correlated features
    corr_features = tsfel.correlated_features(ar_feat_stat)
    ar_feat_stat.drop(corr_features, axis=1, inplace=True)
    
    # Remove low variance features
    selector = VarianceThreshold()
    ar_feat_stat = selector.fit_transform(ar_feat_stat)
    print(ar_feat_stat.shape) 
    
    # Normalize Features
    scaler = preprocessing.StandardScaler()
    ar_feat_stat_norm = scaler.fit_transform(ar_feat_stat)


    cfg_file_temp = tsfel.get_features_by_domain('temporal')                                         
    ar_feat_temp = tsfel.time_series_features_extractor(cfg_file_temp, ar, window_size=window_size)
    corr_features = tsfel.correlated_features(ar_feat_temp)
    ar_feat_temp.drop(corr_features, axis=1, inplace=True)
    selector = VarianceThreshold()
    ar_feat_temp = selector.fit_transform(ar_feat_temp)
    print(ar_feat_temp.shape) 
    scaler = preprocessing.StandardScaler()
    ar_feat_temp_norm = scaler.fit_transform(ar_feat_temp)
    ar_feat_total_norm = np.concatenate([ar_feat_stat_norm,ar_feat_temp_norm],axis=1)
    print(ar_feat_total_norm.shape)

    return ar_feat_total_norm

In [ ]:
ar_KSS_HD_feat_total_norm = create_feat_array("DF_KSS_HD",ar_DF_KSS_ND,1300)
ar_KSS_ND_feat_total_norm = create_feat_array("DF_KSS_ND",ar_DF_KSS_ND,800)